In [ ]:
from ase.io import read
from ase.optimize import FIRE

import numpy as np

In [86]:
structures = read("../../../data/Si-Ge_interfaces_vasp.xyz", index=":")

In [87]:
dft_energies = [structure.get_potential_energy() for structure in structures]

In [88]:
# get lowest energy structure with just Si atoms
Si_slab_idx = -1
Si_slab_energy = float('inf')
Ge_slab_idx = -1
Ge_slab_energy = float('inf')
for idx, structure in enumerate(structures):
    if structure.get_chemical_formula(empirical=True) == "Si":
        if structure.get_potential_energy() < Si_slab_energy:
            Si_slab_energy = structure.get_potential_energy()
            Si_slab_idx = idx
    elif structure.get_chemical_formula(empirical=True) == "Ge":
        if structure.get_potential_energy() < Ge_slab_energy:
            Ge_slab_energy = structure.get_potential_energy()
            Ge_slab_idx = idx
        break

In [89]:
dft_si_slab_energy = dft_energies[Si_slab_idx]
dft_ge_slab_energy = dft_energies[Ge_slab_idx]
print("DFT Si slab energy: ", dft_si_slab_energy)
print("DFT Ge slab energy: ", dft_ge_slab_energy)

DFT Si slab energy:  -401.9679047
DFT Ge slab energy:  -334.78120143


In [90]:
from mace.calculators import mace_mp
from chgnet.model import CHGNetCalculator
mace = mace_mp(model="../../mace-mpa-0-medium.model")
chgnet = CHGNetCalculator()

Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.
CHGNet v0.3.0 initialized with 412,525 parameters


/opt/homebrew/Caskroom/miniconda/base/envs/raffle_env/lib/python3.12/site-packages/mace/calculators/mace.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=m

CHGNet will run on mps


In [91]:
Si_slab = structures[Si_slab_idx].copy()
Si_slab.calc = mace
optimizer = FIRE(Si_slab)
optimizer.run(fmax=0.05, steps=100)
mace_si_slab_energy = Si_slab.get_potential_energy()
print("MACE Si slab energy: ", mace_si_slab_energy)
Ge_slab = structures[Ge_slab_idx].copy()
Ge_slab.calc = mace
optimizer = FIRE(Ge_slab)
optimizer.run(fmax=0.05, steps=100)
mace_ge_slab_energy = Ge_slab.get_potential_energy()
print("MACE Ge slab energy: ", mace_ge_slab_energy)

      Step     Time          Energy          fmax
FIRE:    0 09:43:52     -404.319336        0.299351
FIRE:    1 09:43:52     -404.342407        0.275887
FIRE:    2 09:43:52     -404.378265        0.231842
FIRE:    3 09:43:52     -404.412537        0.172303
FIRE:    4 09:43:53     -404.434326        0.104353
FIRE:    5 09:43:53     -404.442444        0.133328
FIRE:    6 09:43:53     -404.444550        0.172123
FIRE:    7 09:43:54     -404.446014        0.166903
FIRE:    8 09:43:54     -404.448517        0.156692
FIRE:    9 09:43:54     -404.451935        0.141862
FIRE:   10 09:43:54     -404.455872        0.123110
FIRE:   11 09:43:55     -404.459595        0.101140
FIRE:   12 09:43:55     -404.462616        0.076973
FIRE:   13 09:43:55     -404.465088        0.051805
FIRE:   14 09:43:55     -404.466736        0.037868
MACE Si slab energy:  -404.46673583984375
      Step     Time          Energy          fmax
FIRE:    0 09:43:56     -346.881958        0.666953
FIRE:    1 09:43:56     -3

In [92]:
Si_slab = structures[Si_slab_idx].copy()
Si_slab.calc = chgnet
optimizer = FIRE(Si_slab)
optimizer.run(fmax=0.05, steps=100)
chgnet_si_slab_energy = Si_slab.get_potential_energy()
print("CHGNet Si slab energy: ", chgnet_si_slab_energy)
Ge_slab = structures[Ge_slab_idx].copy()
Ge_slab.calc = chgnet
optimizer = FIRE(Ge_slab)
optimizer.run(fmax=0.05, steps=100)
chgnet_ge_slab_energy = Ge_slab.get_potential_energy()
print("CHGNet Ge slab energy: ", chgnet_ge_slab_energy)

      Step     Time          Energy          fmax
FIRE:    0 09:43:59     -409.784050        0.140569
FIRE:    1 09:43:59     -409.792557        0.106247
FIRE:    2 09:43:59     -409.803467        0.068860
FIRE:    3 09:43:59     -409.810371        0.028382
CHGNet Si slab energy:  -409.8103713989258
      Step     Time          Energy          fmax
FIRE:    0 09:43:59     -345.926094        0.255277
FIRE:    1 09:43:59     -345.937805        0.245573
FIRE:    2 09:44:00     -345.959969        0.230151
FIRE:    3 09:44:00     -345.990181        0.200199
FIRE:    4 09:44:00     -346.024323        0.149002
FIRE:    5 09:44:00     -346.059952        0.150131
FIRE:    6 09:44:00     -346.098709        0.165833
FIRE:    7 09:44:00     -346.141357        0.170950
FIRE:    8 09:44:00     -346.193161        0.155809
FIRE:    9 09:44:00     -346.253433        0.119266
FIRE:   10 09:44:00     -346.319084        0.101086
FIRE:   11 09:44:00     -346.387405        0.098362
FIRE:   12 09:44:00     -

In [93]:
for structure in structures:
    structure.calc = mace
mace_energies = [structure.get_potential_energy() for structure in structures]
for structure in structures:
    structure.calc = chgnet
chgnet_energies = [structure.get_potential_energy() for structure in structures]

In [94]:
def get_energy_indices(dft_energies, energies):
    indices = []
    for value in energies:
        diff = np.array(dft_energies) - value
        if np.min(np.abs(diff)) > 0.01:
            print("Warning: no close match found for value", value)
        else:
            indices.append(np.argmin(np.abs(diff)))

    return indices

In [95]:
# get index of the DFT energy closest the value 
aligned_idx = [] # scf, rlx_isif2, rlx_isif3
energies = [ -776.38185986, -777.8058117, -790.5713552 ]
aligned_idx = get_energy_indices(dft_energies, energies)
print("Aligned indices: ", aligned_idx)

Aligned indices:  [4, 47, 27]


In [96]:
energies = [ -776.0978782, -777.5722837, -790.5711185 ]
shift_0_idx = get_energy_indices(dft_energies, energies)

energies = [ -751.1947494 ]
shift_1_idx = get_energy_indices(dft_energies, energies)

energies = [ -765.9984379 ]
shift_2_idx = get_energy_indices(dft_energies, energies)

energies = [ -770.698164 ]
shift_3_idx = get_energy_indices(dft_energies, energies)

energies = [ -735.7029938 ]
shift_4_idx = get_energy_indices(dft_energies, energies)

shift_idx_list = [shift_0_idx, shift_1_idx, shift_2_idx, shift_3_idx, shift_4_idx]

In [97]:
energies = [ -777.6458466, -777.6890089, -790.4285479 ]
mace_0_idx = get_energy_indices(dft_energies, energies)

energies = [ -777.5096264 ]
mace_1_idx = get_energy_indices(dft_energies, energies)

energies = [ -763.614517 ]
chgnet_0_idx = get_energy_indices(dft_energies, energies)

energies = [ -765.4603228 ]
chgnet_1_idx = get_energy_indices(dft_energies, energies)

mace_idx_list = [mace_0_idx, mace_1_idx]
chgnet_idx_list = [chgnet_0_idx, chgnet_1_idx]

In [ ]:
for idx in aligned_idx:
    area = np.linalg.norm(np.cross(structures[idx].get_cell()[0], structures[idx].get_cell()[1]))
    dft_energy_per_area = (dft_energies[idx] - dft_si_slab_energy - dft_ge_slab_energy) / (2 * area)
    mace_energy_per_area = (mace_energies[idx] - mace_si_slab_energy - mace_ge_slab_energy) / (2 * area)
    chgnet_energy_per_area = (chgnet_energies[idx] - chgnet_si_slab_energy - chgnet_ge_slab_energy) / (2 * area)
    print("DFT energy per area: ", dft_energy_per_area, "MACE energy per area: ", mace_energy_per_area, "CHGNet energy per area: ", chgnet_energy_per_area)
    # print in latex table format
    print(f"{dft_energy_per_area:.4f} & {mace_energy_per_area:.4f} & {chgnet_energy_per_area:.4f} \\\\")

DFT energy per area:  -0.16140655563602477 MACE energy per area:  -0.14771244415923873 CHGNet energy per area:  -0.0465756599837699
-0.1614 & -0.1477 & -0.0466 \\
-161.41 & -147.71 & -46.58 \\
DFT energy per area:  -0.1672137047401783 MACE energy per area:  -0.15257074052581093 CHGNet energy per area:  -0.05500353889485115
-0.1672 & -0.1526 & -0.0550 \\
-167.21 & -152.57 & -55.00 \\
DFT energy per area:  -0.213384620196825 MACE energy per area:  -0.18744368449228238 CHGNet energy per area:  -0.10065716105369826
-0.2134 & -0.1874 & -0.1007 \\
-213.38 & -187.44 & -100.66 \\


In [99]:
for idx_list in shift_idx_list:
    for idx in idx_list:
        area = np.linalg.norm(np.cross(structures[idx].get_cell()[0], structures[idx].get_cell()[1]))
        dft_energy_per_area = (dft_energies[idx] - dft_si_slab_energy - dft_ge_slab_energy) / (2 * area)
        mace_energy_per_area = (mace_energies[idx] - mace_si_slab_energy - mace_ge_slab_energy) / (2 * area)
        chgnet_energy_per_area = (chgnet_energies[idx] - chgnet_si_slab_energy - chgnet_ge_slab_energy) / (2 * area)
        print("DFT energy per area: ", dft_energy_per_area, "MACE energy per area: ", mace_energy_per_area, "CHGNet energy per area: ", chgnet_energy_per_area)
        # print in latex table format
        print(f"{dft_energy_per_area:.4f} & {mace_energy_per_area:.4f} & {chgnet_energy_per_area:.4f} \\\\")

DFT energy per area:  -0.1602587213187105 MACE energy per area:  -0.14410824650170034 CHGNet energy per area:  -0.05307298505956199
-0.1603 & -0.1441 & -0.0531 \\
DFT energy per area:  -0.16626309743556564 MACE energy per area:  -0.1517461923755899 CHGNet energy per area:  -0.05428793202495401
-0.1663 & -0.1517 & -0.0543 \\
DFT energy per area:  -0.21341003188800628 MACE energy per area:  -0.18746548327294843 CHGNet energy per area:  -0.10067483207841572
-0.2134 & -0.1875 & -0.1007 \\
DFT energy per area:  -0.058825553238277366 MACE energy per area:  -0.04659927532120463 CHGNet energy per area:  -0.004421661206529005
-0.0588 & -0.0466 & -0.0044 \\
DFT energy per area:  -0.11911521801249442 MACE energy per area:  -0.10244209087936111 CHGNet energy per area:  -0.0347089529228182
-0.1191 & -0.1024 & -0.0347 \\
DFT energy per area:  -0.13825540956519236 MACE energy per area:  -0.12096596156316312 CHGNet energy per area:  -0.04362125697943555
-0.1383 & -0.1210 & -0.0436 \\
DFT energy per ar

In [100]:
for idx_list in mace_idx_list:
    print("MACE structures:")
    for idx in idx_list:
        area = np.linalg.norm(np.cross(structures[idx].get_cell()[0], structures[idx].get_cell()[1]))
        dft_energy_per_area = (dft_energies[idx] - dft_si_slab_energy - dft_ge_slab_energy) / (2 * area)
        mace_energy_per_area = (mace_energies[idx] - mace_si_slab_energy - mace_ge_slab_energy) / (2 * area)
        chgnet_energy_per_area = (chgnet_energies[idx] - chgnet_si_slab_energy - chgnet_ge_slab_energy) / (2 * area)
        print("DFT energy per area: ", dft_energy_per_area, "MACE energy per area: ", mace_energy_per_area, "CHGNet energy per area: ", chgnet_energy_per_area)
        # print in latex table format
        print(f"{dft_energy_per_area:.4f} & {mace_energy_per_area:.4f} & {chgnet_energy_per_area:.4f} \\\\")
for idx_list in chgnet_idx_list:
    print("CHGNet structures:")
    for idx in idx_list:
        area = np.linalg.norm(np.cross(structures[idx].get_cell()[0], structures[idx].get_cell()[1]))
        dft_energy_per_area = (dft_energies[idx] - dft_si_slab_energy - dft_ge_slab_energy) / (2 * area)
        mace_energy_per_area = (mace_energies[idx] - mace_si_slab_energy - mace_ge_slab_energy) / (2 * area)
        chgnet_energy_per_area = (chgnet_energies[idx] - chgnet_si_slab_energy - chgnet_ge_slab_energy) / (2 * area)
        print("DFT energy per area: ", dft_energy_per_area, "MACE energy per area: ", mace_energy_per_area, "CHGNet energy per area: ", chgnet_energy_per_area)
        # print in latex table format
        print(f"{dft_energy_per_area:.4f} & {mace_energy_per_area:.4f} & {chgnet_energy_per_area:.4f} \\\\")

MACE structures:
DFT energy per area:  -0.16655906180772645 MACE energy per area:  -0.15302241492706253 CHGNet energy per area:  -0.05796695301478559
-0.1666 & -0.1530 & -0.0580 \\
DFT energy per area:  -0.16673835048363417 MACE energy per area:  -0.15287624841746653 CHGNet energy per area:  -0.058235732842167716
-0.1667 & -0.1529 & -0.0582 \\
DFT energy per area:  -0.2128369895247581 MACE energy per area:  -0.18749090325323242 CHGNet energy per area:  -0.10293862829774943
-0.2128 & -0.1875 & -0.1029 \\
MACE structures:
DFT energy per area:  -0.1660033138104249 MACE energy per area:  -0.15258664980576694 CHGNet energy per area:  -0.057503657381691095
-0.1660 & -0.1526 & -0.0575 \\
CHGNet structures:
DFT energy per area:  -0.10941020376065742 MACE energy per area:  -0.08863109586402676 CHGNet energy per area:  -0.0628602995591474
-0.1094 & -0.0886 & -0.0629 \\
CHGNet structures:
DFT energy per area:  -0.11692632599218512 MACE energy per area:  -0.09668044578009104 CHGNet energy per area

In [102]:
for idx in aligned_idx:
    area = np.linalg.norm(np.cross(structures[idx].get_cell()[0], structures[idx].get_cell()[1]))
    dft_energy_per_area = (dft_energies[idx] - dft_si_slab_energy - dft_ge_slab_energy) / (2 * area)
    mace_energy_per_area = (mace_energies[idx] - mace_si_slab_energy - mace_ge_slab_energy) / (2 * area)
    chgnet_energy_per_area = (chgnet_energies[idx] - chgnet_si_slab_energy - chgnet_ge_slab_energy) / (2 * area)
    # print as meV per Angstrom^2
    print(f"{dft_energy_per_area * 1000:.2f} & {mace_energy_per_area * 1000:.2f} & {chgnet_energy_per_area * 1000:.2f} \\\\")
for idx_list in shift_idx_list:
    for idx in idx_list:
        area = np.linalg.norm(np.cross(structures[idx].get_cell()[0], structures[idx].get_cell()[1]))
        dft_energy_per_area = (dft_energies[idx] - dft_si_slab_energy - dft_ge_slab_energy) / (2 * area)
        mace_energy_per_area = (mace_energies[idx] - mace_si_slab_energy - mace_ge_slab_energy) / (2 * area)
        chgnet_energy_per_area = (chgnet_energies[idx] - chgnet_si_slab_energy - chgnet_ge_slab_energy) / (2 * area)
        # print as meV per Angstrom^2
        print(f"{dft_energy_per_area * 1000:.2f} & {mace_energy_per_area * 1000:.2f} & {chgnet_energy_per_area * 1000:.2f} \\\\")
for idx_list in mace_idx_list:
    for idx in idx_list:
        area = np.linalg.norm(np.cross(structures[idx].get_cell()[0], structures[idx].get_cell()[1]))
        dft_energy_per_area = (dft_energies[idx] - dft_si_slab_energy - dft_ge_slab_energy) / (2 * area)
        mace_energy_per_area = (mace_energies[idx] - mace_si_slab_energy - mace_ge_slab_energy) / (2 * area)
        chgnet_energy_per_area = (chgnet_energies[idx] - chgnet_si_slab_energy - chgnet_ge_slab_energy) / (2 * area)
        # print as meV per Angstrom^2
        print(f"{dft_energy_per_area * 1000:.2f} & {mace_energy_per_area * 1000:.2f} & {chgnet_energy_per_area * 1000:.2f} \\\\")
for idx_list in chgnet_idx_list:
    for idx in idx_list:
        area = np.linalg.norm(np.cross(structures[idx].get_cell()[0], structures[idx].get_cell()[1]))
        dft_energy_per_area = (dft_energies[idx] - dft_si_slab_energy - dft_ge_slab_energy) / (2 * area)
        mace_energy_per_area = (mace_energies[idx] - mace_si_slab_energy - mace_ge_slab_energy) / (2 * area)
        chgnet_energy_per_area = (chgnet_energies[idx] - chgnet_si_slab_energy - chgnet_ge_slab_energy) / (2 * area)
        # print as meV per Angstrom^2
        print(f"{dft_energy_per_area * 1000:.2f} & {mace_energy_per_area * 1000:.2f} & {chgnet_energy_per_area * 1000:.2f} \\\\")

-161.41 & -147.71 & -46.58 \\
-167.21 & -152.57 & -55.00 \\
-213.38 & -187.44 & -100.66 \\
-160.26 & -144.11 & -53.07 \\
-166.26 & -151.75 & -54.29 \\
-213.41 & -187.47 & -100.67 \\
-58.83 & -46.60 & -4.42 \\
-119.12 & -102.44 & -34.71 \\
-138.26 & -120.97 & -43.62 \\
4.27 & 11.43 & 31.39 \\
-166.56 & -153.02 & -57.97 \\
-166.74 & -152.88 & -58.24 \\
-212.84 & -187.49 & -102.94 \\
-166.00 & -152.59 & -57.50 \\
-109.41 & -88.63 & -62.86 \\
-116.93 & -96.68 & -62.86 \\


In [104]:
from ase.visualize import view
view(SiGe_abrupt)

<Popen: returncode: None args: ['/opt/homebrew/Caskroom/miniconda/base/envs/...>